# Домашнее задание 2. Извлечение коллокаций.

При выполнении домашнего задания можно пользоваться тетрадками с семинаров.

### Описание задания:

1. Скачайте [корпус](https://github.com/sjut/HSE-Compling/blob/master/hw/testset2.txt) текстов, обработайте его с помощью UDPipe, извлеките все группы 
"глагол + прямое дополнение, выраженное существительным" 
(не учитывайте глаголы, которые встречаются в корпусе менее **50** раз).

2. Оцените полученные словосочетания следующими метриками: *log-likelihood*, *dice*, *PMI* (можно использовать `nltk.collocations`). 

3. Подготовьте "золотой стандарт" коллокаций (далее ЗС) для этого корпуса: 
возьмите словосочетания, которые попадают в топ-100 по всем метрикам,
пересеките со [словарем глагольной сочетаемости](https://yadi.sk/d/5WWwOr9ccemcZA).

4. Добавьте в ЗС словосочетания из топ-100 , которые не вошли в словарь, но являются коллокациями (если такие есть), объясните свой выбор.

5. Оцените ранговую корреляцию (коэффициент Спирмена) результатов по каждой метрике с ЗС. 
Как это работает, читайте, например, [тут](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient#Example).
Можно использовать `scipy.stats.spearmanr`.
Опишите ошибки каждой метрики.

### Критерии оценки:

По 2 балла на каждый пункт.

### Формат сдачи задания:

Jupyter-notebook на гитхабе.

### Дедлайн: 

26 ноября 2019 10:00мск

In [8]:
import nltk

from collections import Counter
from itertools import chain
from nltk import collocations
from nltk.parse import DependencyGraph
from nltk.tokenize import RegexpTokenizer
from pymorphy2 import MorphAnalyzer
from scipy.stats import spearmanr
from collections import Counter

In [9]:
morph = MorphAnalyzer()
tokenizer = RegexpTokenizer(r'\w+')

In [116]:
def normalize_text(text):
    text = tokenizer.tokenize(text.lower())
    lemmas = [morph.parse(t)[0].normal_form for t in text]
    return ' '.join(lemmas)

## 1. Скачиваем модель

In [1]:
!/Users/Asalamatina/Desktop/udpipe-1.2.0-bin/bin-osx/udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
/Users/Asalamatina/Desktop/russian-syntagrus-ud-2.4-190531.udpipe \
< /Users/Asalamatina/Desktop/corpus.txt > /Users/Asalamatina/Desktop/corpus.conllu

Loading UDPipe model: done.


## 2. Извлекаем глаголы

In [118]:
trees = []

with open('corpus.conllu', 'r', encoding='utf-8') as f:
    parsed_sents = f.read().split('\n\n')

    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))


## 3. Глагол + прямое дополнение, выраженное существительным

In [119]:
verb_vocab = Counter()
for t in trees:
    try:
        g = DependencyGraph(t, top_relation_label='root')
        for n in g.nodes:
            if g.nodes[n]['ctag'] == 'VERB':
                verb_vocab[g.nodes[n]['lemma']] += 1
    except:
        pass
mostfrequent = [item[0] for item in verb_vocab.most_common() if item[1] >= 50]
print(mostfrequent)

['обвинить', 'мочь', 'признать', 'подать', 'заявить', 'сообщить', 'стать', 'удовлетворить', 'требовать', 'вынести', 'считать', 'принять', 'просить', 'обратиться', 'говорить', 'быть', 'получить', 'приговорить', 'обвинять', 'отказаться', 'находиться', 'являться', 'направить', 'обжаловать', 'рассматривать', 'сказать', 'напомнить', 'предъявить', 'иметь', 'подтвердить', 'рассмотреть', 'пытаться', 'решить', 'отменить', 'арестовывать', 'делать', 'начаться', 'объявить', 'утверждать', 'дать', 'отклонить', 'передать', 'нет', 'оспорить', 'назначить', 'провести', 'согласиться', 'взыскать', 'оказаться', 'удаться', 'оставить', 'отказать', 'выплатить', 'обязать', 'пояснить', 'использовать', 'доказать', 'связать', 'счесть', 'запретить', 'рассказать', 'постановить', 'принадлежать', 'оспаривать', 'пройти']


In [120]:
bigrams = []
for t in trees:
    try:
        g = DependencyGraph(t, top_relation_label='root')
        for item in g.triples():
            if item[1] == 'obj' and item[2][1] == 'NOUN':
                lemma = normalize_text(item[0][0])
                if lemma in mostfrequent:
                    coll = tuple([normalize_text(item[0][0]), 
                                  normalize_text(item[2][0])])
                    bigrams.append(coll)
    except:
        pass

In [121]:
from nltk.collocations import *
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Asalamatina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 4. Оценка метриками log-likelihood, dice, PMI 

In [122]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(bigrams)
finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in nltk.corpus.stopwords.words('russian'))

In [123]:
loglike_scores = {i[0]:i[1] for i in finder.score_ngrams(bigram_measures.likelihood_ratio)}
pmi_scores = {i[0]:i[1] for i in finder.score_ngrams(bigram_measures.pmi)}
dice_scores = {i[0]:i[1] for i in finder.score_ngrams(bigram_measures.dice)}

In [124]:
pmi_100 = finder.nbest(bigram_measures.pmi, 100)
loglike_100 = finder.nbest(bigram_measures.likelihood_ratio, 100)
dice_100 =  finder.nbest(bigram_measures.dice, 100)

## 5. Золотой стандарт

In [125]:
metrics = set(pmi_100best).intersection(set(dice_100best)).intersection(set(loglike_100best))
metrics

{('арестовывать', 'акция'),
 ('вынести', 'вердикт'),
 ('вынести', 'приговор'),
 ('вынести', 'решение'),
 ('выплатить', 'дивиденд'),
 ('выплатить', 'иркутянин'),
 ('выплатить', 'компенсация'),
 ('выплатить', 'штраф'),
 ('дать', 'возможность'),
 ('дать', 'заключение'),
 ('дать', 'определение'),
 ('дать', 'показание'),
 ('дать', 'указание'),
 ('делать', 'вид'),
 ('доказать', 'бездействие'),
 ('доказать', 'вина'),
 ('доказать', 'невиновность'),
 ('доказать', 'незаконность'),
 ('доказать', 'неправомерность'),
 ('доказать', 'правность'),
 ('запретить', 'деятельность'),
 ('иметь', 'право'),
 ('использовать', 'женщинсмертница'),
 ('назначить', 'наказание'),
 ('направить', 'альтернативщик'),
 ('направить', 'запрос'),
 ('обвинить', 'президент'),
 ('обжаловать', 'приговор'),
 ('обжаловать', 'решение'),
 ('обжаловать', 'санкция'),
 ('объявить', 'перерыв'),
 ('обязать', 'ответчик'),
 ('оспаривать', 'законность'),
 ('оспаривать', 'отказ'),
 ('оспаривать', 'предписание'),
 ('оспаривать', 'решение'),


In [126]:
verb_coll = []
with open("verb_coll.txt", 'r', encoding="utf-8") as f:
    text = f.read()

lines = text.split("\n")
for line in lines:
    entry = line.split("\t")
    if len(entry) > 2:
        coll = entry[2]
        words = coll.split()
        if len(words) == 2:
            coll_lemm = (m.parse(words[0])[0].normal_form, m.parse(words[1])[0].normal_form)
            verb_coll.append(coll_lemm)

In [127]:
golden_standart = metrics.intersection(set(verb_coll))
golden_standart

{('вынести', 'приговор'),
 ('вынести', 'решение'),
 ('дать', 'возможность'),
 ('дать', 'заключение'),
 ('дать', 'определение'),
 ('дать', 'показание'),
 ('дать', 'указание'),
 ('делать', 'вид'),
 ('доказать', 'вина'),
 ('направить', 'запрос'),
 ('обжаловать', 'приговор'),
 ('обжаловать', 'решение'),
 ('объявить', 'перерыв'),
 ('оспаривать', 'решение'),
 ('отменить', 'постановление'),
 ('отменить', 'решение'),
 ('подать', 'жалоба'),
 ('подать', 'иск'),
 ('получить', 'контроль'),
 ('предъявить', 'обвинение'),
 ('предъявить', 'претензия'),
 ('принять', 'мера'),
 ('принять', 'резолюция'),
 ('принять', 'решение'),
 ('пройти', 'курс'),
 ('пройти', 'проверка'),
 ('рассматривать', 'вопрос'),
 ('рассматривать', 'дело'),
 ('рассмотреть', 'вопрос'),
 ('рассмотреть', 'жалоба'),
 ('решить', 'вопрос'),
 ('решить', 'проблема'),
 ('решить', 'судьба'),
 ('удовлетворить', 'запрос'),
 ('удовлетворить', 'иск')}

In [128]:
len(golden_standart)

35

In [129]:
metrics - golden_standart

{('арестовывать', 'акция'),
 ('вынести', 'вердикт'),
 ('выплатить', 'дивиденд'),
 ('выплатить', 'иркутянин'),
 ('выплатить', 'компенсация'),
 ('выплатить', 'штраф'),
 ('доказать', 'бездействие'),
 ('доказать', 'невиновность'),
 ('доказать', 'незаконность'),
 ('доказать', 'неправомерность'),
 ('доказать', 'правность'),
 ('запретить', 'деятельность'),
 ('иметь', 'право'),
 ('использовать', 'женщинсмертница'),
 ('назначить', 'наказание'),
 ('направить', 'альтернативщик'),
 ('обвинить', 'президент'),
 ('обжаловать', 'санкция'),
 ('обязать', 'ответчик'),
 ('оспаривать', 'законность'),
 ('оспаривать', 'отказ'),
 ('оспаривать', 'предписание'),
 ('оспорить', 'доначисление'),
 ('оспорить', 'правомерность'),
 ('оспорить', 'предписание'),
 ('оспорить', 'претензия'),
 ('оспорить', 'решение'),
 ('оспорить', 'указ'),
 ('оставить', 'приговор'),
 ('отклонить', 'иск'),
 ('отменить', 'запрет'),
 ('отменить', 'определение'),
 ('отменить', 'регистрация'),
 ('отменить', 'указ'),
 ('подать', 'апелляция'),
 

## 6. Не попали в ЗС

Мне кажется, что в ЗС нужно добавить следующие коллокации:
 1. иметь право
 2. предъявить ультиматум 
 3. выплатить компенсациию
 
    

In [130]:
add_back = {('иметь', 'право'),
            ('предъявить', 'ультиматум'),
            ('выплатить', 'компенсация')}

In [131]:
golden_standart = golden_standart.union(add_back)
len(golden_standart)

38

## 7. Ранговая корреляция

In [132]:
rho, pval = spearmanr([1, 2, 3], [10, 8, 11])
pval

0.6666666666666667